In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
path = "./Datasets/"

In [3]:
log_df = pd.read_parquet(path + "logs.parquet")

In [4]:
log_df.head(10)

,resp_time,bytes_sent,resp_code,url,datetime,app_1_containers,app_2_containers,app_3_containers,app_4_containers
0,0.031,197,200,/2/4-3/65,2019-09-05 22:37:10,3,3,3,3
1,0.380,198,200,/3/3-4/37,2019-09-05 22:37:10,3,3,3,3
2,0.298,188,200,/4/3/188,2019-09-05 22:37:11,3,3,3,3
3,0.009,188,200,/3/3/180,2019-09-05 22:37:11,3,3,3,3
4,0.331,199,200,/3/3-4/115,2019-09-05 22:37:11,3,3,3,3
5,0.148,229,200,/3/3-3-3-3-3/101,2019-09-05 22:37:11,3,3,3,3
6,0.023,188,200,/3/3/180,2019-09-05 22:37:11,3,3,3,3
7,0.298,189,200,/4/3/188,2019-09-05 22:37:11,3,3,3,3
8,0.027,207,200,/3/4-2-2/38,2019-09-05 22:37:11,3,3,3,3
9,0.352,210,200,/3/4-4-3/60,2019-09-05 22:37:11,3,3,3,3


In [ ]:
# Feature transformations

# Convert resp_time and bytes_setn into float

log_df.resp_time = log_df.resp_time.astype("float")
log_df.bytes_sent = log_df.bytes_sent.astype("float")

# Convert datetime string into Pandas Datetime

log_df.datetime = pd.to_datetime(log_df.datetime,
                                    format="%Y-%m-%d %H:%M:%S")

# Create categorical variables

log_df = pd.get_dummies(log_df, columns = ['resp_code'])
log_df = pd.get_dummies(log_df, columns = ['url'])


In [ ]:
log_df.tail(10)

In [ ]:
log_df.info()

In [ ]:
log_df.shape

In [ ]:
# EDA response time

log_df.resp_time.hist(bins=20)

# Compare response time where all apps where replicated 3X to where only one was

log_df["all_x_3"] = (log_df.app_1_containers == 3) & (log_df.app_2_containers == 3) & (log_df.app_3_containers == 3) & (log_df.app_4_containers == 3)

log_df["2_x_3"] = (log_df.app_1_containers == 1) & (log_df.app_2_containers == 3) & (log_df.app_3_containers == 1) & (log_df.app_4_containers == 1)

log_df["3_x_3"] = (log_df.app_1_containers == 1) & (log_df.app_2_containers == 1) & (log_df.app_3_containers == 3) & (log_df.app_4_containers == 1)

log_df["3_x_3"] = (log_df.app_1_containers == 1) & (log_df.app_2_containers == 1) & (log_df.app_3_containers == 3) & (log_df.app_4_containers == 1)

log_df["4_x_3"] = (log_df.app_1_containers == 1) & (log_df.app_2_containers == 1) & (log_df.app_3_containers == 1) & (log_df.app_4_containers == 4)

log_df["23_x_6"] = (log_df.app_1_containers == 2) & (log_df.app_2_containers == 6) & (log_df.app_3_containers == 6) & (log_df.app_4_containers == 2)

log_df["2_x_3"].value_counts()

log_df["3_x_3"].value_counts()

log_df["4_x_3"].value_counts()

log_df["23_x_6"].value_counts()

log_df["all_x_3"].value_counts()

log_df[log_df["all_x_3"] == True].resp_time.hist(bins=20)

log_df[log_df["2_x_3"] == True].resp_time.hist(bins=20)

log_df[log_df["3_x_3"] == True].resp_time.hist(bins=20)

log_df[log_df["4_x_3"] == True].resp_time.hist(bins=20)

log_df[log_df["23_x_6"] == True].resp_time.hist(bins=20)

In [ ]:
# What about errors

(log_df.resp_time.isna().sum() / (len(log_df))) * 100

# 5% error rate overall. What is the distribution over container patterns?

In [ ]:
log_df.columns.tolist()